In [3]:
pip install tensorflow

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 MB 530.2 kB/s eta 0:00:00m eta 0:00:010:00:11
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 247.6 kB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 792.0 kB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 888.1 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 602.3 kB/s eta 0:00:0031m1.3 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 735.9 kB/s eta 0:00:000:00:010:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 395.2 kB/s eta 0:00:001m448.2 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
pip install sklearn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 639.5 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 928.6 kB/s eta 0:00:00m eta 0:00:010:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1304 sha256=31e3f0de20db9105049cc8ccd8f61e6540bfa62ff4a781a88299a079aa13b99c
  Stored in directory: /Users/davinahn/Library/Caches/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see 

In [3]:
import os
from scipy.io import loadmat
import time

import pandas as pd
import numpy as np

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.random import set_seed

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae, r2_score as r2
from sklearn.inspection import permutation_importance

import matplotlib.pyplot as plt

In [4]:
np.random.seed(1337)
set_seed(1337)

In [5]:
def sape(y_true, y_pred):
    return 200*(np.abs(y_pred-y_true)/(np.abs(y_true) + np.abs(y_pred)))

def relative_directive_error(y_true, y_pred, threshold):
    y_true_sort = np.sort(y_true)
    y_pred_sort = np.sort(y_pred)
    y_true_threshold = np.abs(y_true_sort - threshold).argmin()
    rel_error = np.abs(y_pred_sort[y_true_threshold]-y_true_sort[y_true_threshold])/threshold*100
    return rel_error

def rolling_day_agg(data, start, end, target, same_day=True):
    if same_day:
        comb = data[(data["zeit"] >= start) & (data["zeit"] <= end)]. \
                groupby("datum", as_index=False).agg({target: "mean"})
    else:
        prev_day = data.loc[(data["zeit"] >= start)].groupby("datum", as_index=False).agg({target: ["mean", "size"]})
        prev_day.columns = ["_".join(col_name).rstrip("_") for col_name in prev_day.columns.to_flat_index()]
        next_day = data.loc[(data["zeit"] <= end)].groupby("datum", as_index=False).agg({target: ["mean", "size"]})
        next_day.columns = ["_".join(col_name).rstrip("_") for col_name in next_day.columns.to_flat_index()]
        
        comb = prev_day.merge(next_day, on="datum")
        comb[[target+"_mean_x", target+"_size_x"]] = comb[[target+"_mean_x", target+"_size_x"]].shift(1)
        comb[target] = (comb[target+"_mean_x"] * comb[target+"_size_x"] + comb[target+"_mean_y"] * comb[target+"_size_y"]) \
                       / (comb[target+"_size_x"] + comb[target+"_size_y"])
        
    return comb[["datum", target]]

def feature_engine(data):
    # all times in UTC!
    data_train = data.groupby("datum", as_index=False).agg({"pm": "mean"})
    data_train["pmshift"] = data_train["pm"].shift(1)

    data_train["regen"] = data.groupby("datum", as_index=False).agg({"regen":"mean"})["regen"]
    data_train = data_train.merge(rolling_day_agg(data, "07:00", "12:00", "regen"), on="datum", 
                                  how='left', suffixes=("", "1"))
    data_train = data_train.merge(rolling_day_agg(data, "12:30", "16:00", "regen"), on="datum", 
                                  how='left', suffixes=("", "2"))
    data_train = data_train.merge(rolling_day_agg(data, "16:30", "06:30", "regen", same_day=False), 
                                  on="datum", how='left', suffixes=("", "3"))

    data_train = data_train.merge(rolling_day_agg(data, "05:00", "17:00", "windGe"), on="datum", 
                                  how='left', suffixes=("", "1"))
    data_train = data_train.merge(rolling_day_agg(data, "16:30", "06:30", "windGe", same_day=False), 
                                  on="datum", how='left', suffixes=("", "2"))
    
    temp1 = rolling_day_agg(data, "05:00", "17:00", "temp")
    temp2 = rolling_day_agg(data, "17:30", "04:30", "temp", same_day=False)
    temp1["tdiff1"] = temp1["temp"].diff()
    temp2["tdiff2"] = temp2["temp"].diff()
    
    data_train = data_train.merge(temp1[["datum", "tdiff1"]], on="datum", how='left')
    data_train = data_train.merge(temp2[["datum", "tdiff2"]], on="datum", how='left')
    
    maxtemp1 = data[(data["zeit"].str.startswith("04:00"))][["datum", "temp"]]
    mintemp2 = data[(data["zeit"].str.startswith("12:00"))][["datum", "temp"]]
    deltatemp = maxtemp1.merge(mintemp2, on="datum", suffixes=("max", "min"))
    deltatemp["temp_grad"] = deltatemp["tempmin"] - deltatemp["tempmax"]
    
    data_train = data_train.merge(deltatemp[["datum", "temp_grad"]], on="datum", how='left')
    data_train["temp"] = data.groupby("datum", as_index=False).agg({"temp":"mean"})["temp"]
    
    data_train["wochtag"] = data.groupby(["datum"], as_index=False).agg({"wochtag": "mean"})["wochtag"]
    data_train = data_train.merge(data.groupby(["wochtag"], as_index=False).agg({"pm": "mean"}), 
                                  on="wochtag", suffixes=("", "mean"))
    data_train["wochtag"] = data_train["pmmean"]
    data_train.drop(columns="pmmean", inplace=True)
    data_train = data_train.sort_values("datum")
    
    data_train = data_train.dropna().reset_index(drop=True)
    
    return data_train

def get_model(width = 64, depth = 2, loss="mean_squared_error", len_input=32):
    one_input = Input(shape=(len_input,), name='one_input')
    #x = Dense(8, activation="relu", kernel_initializer='uniform')(one_input)
    x = Dense(width, activation="linear", kernel_initializer='uniform')(one_input)
    
    for i in range(1,depth):
        #width = max(8, int(width/2))
        x = Dense(width, activation="relu", kernel_initializer='uniform')(x)
        x = Dropout(0.2)(x)
        
    x = Dense(1, kernel_initializer='uniform', name="main_output", activation="linear")(x)
    
    model = Model(inputs=one_input, outputs=x)
    model.compile(loss=loss, optimizer='adam')
    return model

In [6]:
columns = {
    "pm25": "pm"
}

data = {}
data_valid = {}
data_train = {}
data_test = {}

for dirpath,_ , files in os.walk("data-full/"):
    for f in files:
        data_raw = pd.read_csv(dirpath + f)
        data_raw["timestamp"] = pd.to_datetime(data_raw["timestamp"])
        data_raw["zeit"] = data_raw["timestamp"].dt.time.astype(str)
        data_raw["datum"] = data_raw["timestamp"].dt.date.astype(str)
        data_raw["wochtag"] = np.maximum(data_raw["timestamp"].dt.weekday - 3, 1)
        for sensor in data_raw["sensor"].unique():
            if "train" in f:
                data[sensor] = data_raw[data_raw["sensor"] == sensor].copy().rename(columns=columns)
                data_train[sensor] = feature_engine(data[sensor])
            else:
                data_valid[sensor] = data_raw[data_raw["sensor"] == sensor].copy().rename(columns=columns)
                data_test[sensor] = feature_engine(data_valid[sensor])

In [10]:
# NNs
train_predict = {}
test_predict = {}

for station in data_test.keys():
    if True:
        features = list(range(2,14))
        X_train = data_train[station].iloc[:, features].values
        y_train = data_train[station].loc[:, "pm"].values

        X_test = data_test[station].iloc[:, features].values
        y_test = data_test[station].loc[:, "pm"].values

        model = get_model(10, 1, len_input=len(X_train[0]))
        start = time.time()
        model.fit(X_train, y_train, shuffle=True, batch_size=16, epochs=200, verbose=0)
        end = time.time() - start

        mse_train = model.evaluate(X_train, y_train, verbose=0)
        mse_valid = model.evaluate(X_test, y_test, verbose=0)
        
        train_predict[station] = model.predict(X_train)[:,0]
        test_predict[station] = model.predict(X_test)[:,0]
        
        mae_train = mae(y_train, train_predict[station])
        mae_valid = mae(y_test, test_predict[station])

        smape_train = sape(y_train, train_predict[station]).mean()
        smape_valid = sape(y_test, test_predict[station]).mean()
        
        r2_train = r2(y_train, train_predict[station])
        r2_valid = r2(y_test, test_predict[station])
        
        rde_valid = relative_directive_error(y_test, test_predict[station], threshold=15)

        #ps = permutation_importance(model, X_test, y_test, scoring="neg_mean_squared_error")["importances_mean"]

        K.clear_session()
        print("{}: MSE: {:.2f}/{:.2f} RMSE: {:.2f}/{:.2f} MAE: {:.2f}/{:.2f} SMAPE: {:.2f}%/{:.2f}% R²: {:.2f}/{:.2f} RDE: {:.2f}% in {:.1f}s".format(
                                                                    station, mse_train, mse_valid, 
                                                                    np.sqrt(mse_train), np.sqrt(mse_valid),
                                                                    mae_train, mae_valid,
                                                                    smape_train, smape_valid,
                                                                    r2_train, r2_valid,
                                                                    rde_valid, end))
        #print(" ".join(["{}".format(x) for x in data_train[station].columns[features]])) 
        #print("   ".join(["{:.2f}".format(y) for y in ps]))

In [11]:
plot_station = "215"
plot_data = data_test[plot_station].copy()
plot_data["datum"] = pd.to_datetime(plot_data["datum"])
plot_data["pmp"] = test_predict[plot_station]
plot_data = pd.DataFrame(plot_data.resample("d", on="datum")[["datum", "pm", "pmp"]].first())

fig, ax = plt.subplots(figsize=(16,3))

ax.plot(plot_data["pm"], label="PM2.5")
ax.plot(plot_data["pmp"], label="prediction")
ax.set_title(plot_station.capitalize(), size=16)
ax.set_ylabel('PM2.5', size=20)
ax.grid()

ax.set_ylim(0, plot_data["pm"].max() + 20)
#ax.set_xlim("2021-09", "2022-04")
ax.tick_params(axis='x', pad=10)

for item in ax.get_xticklabels() + ax.get_yticklabels():
    item.set_size(16)
    
ax.legend(fontsize=16, framealpha=1.0)

KeyError: '215'